## Definition Generation

### Prompt Generation

In [87]:
system_role_prompt = "Act as a Lawyer drafting European Legislative documents to be published on Eur-Lex website."

In [88]:
system_instruction_prompt = "Define the term: {term}, based on the sentences provided between the triple dashes where different sentences are splitted by new line character \n. ---{sentences}---"

In [89]:
system_context_prompt = "Provide a clear and concise definition strictly within 35 to 50 words that accurately conveys the meaning within the context of the sentences."

In [90]:
system_output_prompt = """Give your output in JSON format with following keys: [term, definition] and definition must be strictly in the format "'term' means". Just return the JSON, do not add ANYTHING, NO INTERPRETATION!"""

In [91]:
llama_template_1 = f"""
<s>[INST]<<SYS>>
{system_role_prompt}\n
{system_instruction_prompt}\n
{system_context_prompt}\n
{system_output_prompt}<</SYS>>
[/INST]
"""

In [92]:
llama_template_2 = f"""
[INST] <<SYS>>
{system_role_prompt}\n
{system_context_prompt}\n
{system_output_prompt}
<</SYS>>

{system_instruction_prompt}\n
[/INST]
"""

In [93]:
llama_template_3 = f"""
[INST] <<SYS>>
{system_role_prompt}\n
{system_instruction_prompt}\n
{system_context_prompt}\n
<</SYS>>

{system_output_prompt}
[/INST]
"""

In [94]:
term = "energy infrastructure bottleneck"
sentences = """The following specific criteria shall apply to projects of common interest falling within specific energy infrastructure categories: (a) for electricity transmission, distribution and storage projects falling under the energy infrastructure categories set out in point (1)(a), (b), (c), (d) and (f) of Annex II, the project contributes significantly to sustainability through the integration of renewable energy into the grid, the transmission or distribution of renewable generation to major consumption centres and storage sites, and to reducing energy curtailment, where applicable, and contributes to at least one of the following specific criteria:(i)market integration, including through lifting the energy isolation of at least one Member State and reducing energy infrastructure bottlenecks, competition, interoperability and system flexibility;(ii)security of supply, including through interoperability, system flexibility, cybersecurity, appropriate connections and secure and reliable system operation;"""

In [2]:
# Common required libraries
!pip install -q transformers einops accelerate langchain bitsandbytes sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00


### LLAMA-2

#### References:
1. [Get the LLAMA-2 model](https://levelup.gitconnected.com/text-summarization-llama2-how-to-use-llama2-with-langchain-ad5775c80716)
2. [Insights about Prompting](https://medium.com/@sasika.roledene/unlocking-llm-fundamental-of-prompt-engineering-with-llama-2-ee8649552115)
3. [LLAMA-2 prompting](https://huggingface.co/blog/llama2#how-to-prompt-llama-2)


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from langchain import PromptTemplate,  LLMChain

In [5]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
llm = HuggingFacePipeline(pipeline = pipeline,
                          model_kwargs = {'temperature':0}
                          )

In [96]:
prompt = PromptTemplate(template=llama_template_2, input_variables=["term", "sentences"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
llama_response_2 = llm_chain.run({"term": term, "sentences": sentences})

In [97]:
llama_response_2

'{\n"term": "energy infrastructure bottleneck",\n"definition": " \'energy infrastructure bottleneck\' means any obstacle, constraint or limitation in the transmission, distribution or storage of energy, including but not limited to: (i) lack of interoperability, system flexibility or cybersecurity; (ii) inadequate connections or secure and reliable system operation; (iii) energy isolation or curtailment; (iv) lack of competition or market integration; (v) any other factor that hinders the efficient and sustainable functioning of the energy infrastructure."\n}'

In [98]:
prompt = PromptTemplate(template=llama_template_1, input_variables=["term", "sentences"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
llama_response_1 = llm_chain.run({"term": term, "sentences": sentences})

In [99]:
llama_response_1

'{\n"term": "energy infrastructure bottleneck",\n"definition": "a limitation or obstacle in the transmission, distribution, or storage of energy, particularly in the integration of renewable energy sources into the grid, resulting in reduced efficiency, increased costs, or decreased reliability."\n}'

In [100]:
prompt = PromptTemplate(template=llama_template_3, input_variables=["term", "sentences"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
llama_response_3 = llm_chain.run({"term": term, "sentences": sentences})

In [101]:
llama_response_3

'{\n"term": "energy infrastructure bottleneck",\n"definition": "\'Energy infrastructure bottleneck\' means a limitation or obstacle in the transmission, distribution or storage of energy, including but not limited to: (i) physical constraints such as pipeline capacity, grid connectivity, or storage capacity; (ii) regulatory or administrative barriers such as lack of coordination or inconsistent policies; (iii) financial constraints such as lack of investment or funding; or (iv) technical constraints such as interoperability issues or cybersecurity risks."\n}'

**LLAMA_TEMPLATE_1** is providing a precise definition and not an ambiguous definition

In [ ]:
response_output = {}
falcon_response = " "
chatgpt_response = " "

In [ ]:
response_output[term] = {
    "term": term,
    "sentences": sentences,
    "llama_generated_definition": llama_response_1,
    "falcon_generated_definition": falcon_response,
    "openai_generated_definition": chatgpt_response,
}

## Definition Comparison

### Prompt Generation

In [70]:
system_role_prompt = "Act as a Lawyer drafting European Legislative documents to be published on Eur-Lex website."

In [71]:
system_instruction_prompt = "You have a definition of {term} as {definition}"

In [72]:
system_context_prompt = "Verify if the term '[term]' holds the definition '[definition]' in the context of the current sentence: ---{sentences}---."

In [81]:
system_example = """Examples:
term: "importer"
definition: ‘importer’ means any natural or legal person established within the Union who places a product from a third country on the Union market;
statements:
---
‘supplier’ means a manufacturer established in the Union, an authorised representative of a manufacturer who is not established in the Union, or an importer, who places a product on the Union market;
---
output: True

term: "manufacturer"
definition: ‘manufacturer’ means any natural or legal person who manufactures a product or has a product designed or manufactured, and markets that product under its name or trademark;
statements:
---
Traffic noise is a significant nuisance and has a harmful effect on health. Regulation (EC) No 661/2009 lays down minimum requirements for the external rolling noise of tyres. Technological developments make it possible to reduce external rolling noise significantly beyond those minimum requirements. To reduce traffic noise, it is therefore appropriate to update the provisions on the labelling of tyres to encourage manufacturer to purchase tyres with lower external rolling noise by providing them with harmonised information on the external rolling noise parameter.
---
output: False
"""

In [82]:
system_output_prompt = """Give your output as a binary value of true or false where True when term holds the definition in the provided sentences otherwise False. Just return the JSON, do not add ANYTHING, NO INTERPRETATION!"""

In [83]:
llama_template_term_comparison = f"""
<s>[INST]<<SYS>>
{system_role_prompt}\n
{system_instruction_prompt}\n
{system_context_prompt}\n
{system_example}\n
{system_output_prompt}
<</SYS>>
[/INST]
"""

In [84]:
llama_template_term_comparison

'\n<s>[INST]<<SYS>>\nAct as a Lawyer drafting European Legislative documents to be published on Eur-Lex website.\n\nYou have a definition of {term} as {definition}\n\nVerify if the term \'[term]\' holds the definition \'[definition]\' in the context of the current sentence: ---{sentences}---.\n\nExample:\nterm: "importer"\ndefinition: ‘importer’ means any natural or legal person established within the Union who places a product from a third country on the Union market;\nstatements: \n---\n‘supplier’ means a manufacturer established in the Union, an authorised representative of a manufacturer who is not established in the Union, or an importer, who places a product on the Union market;\n---\noutput: True\n\n\nGive your output as a binary value of true or false where True when term holds the definition in the provided sentences otherwise False. Just return the JSON, do not add ANYTHING, NO INTERPRETATION!\n<</SYS>>\n[/INST]\n'

In [62]:
definition_comparison_prompt = """
[INST]
<<SYS>> Act as a Lawyer drafting European Legislative documents to be published on Eur-Lex website.
Given a term definition and statements, verify if the term definition holds true in the context of the statements between three dashed lines.
If term definitions hold true in statements, provide output as a binary value "true".
If term definitions do not hold true in statements, provide output as a binary value "false"

For example,
term: "importer"
definition: ‘importer’ means any natural or legal person established within the Union who places a product from a third country on the Union market;
statements:
---
‘supplier’ means a manufacturer established in the Union, an authorised representative of a manufacturer who is not established in the Union, or an importer, who places a product on the Union market;
---
output: True

User Input:
term: {term}
definition: {definition}
statements:
---
{sentences}
---
<<\SYS>>

ONLY_OUTPUT, do not add ANYTHING, NO INTERPRETATION!
[\INST]
"""


In [54]:
definition_comparison_prompt= """
[INST]
<<SYS>> Act as a Lawyer drafting European Legislative documents to be published on Eur-Lex website.
Given a term definition and statements, verify if the term definition holds true in the context of the statements between three dashed lines.
If term definitions hold true in statements, provide output as a binary value "true".
If term definitions do not hold true in statements, provide output as a binary value "false"

For example, term: "importer" definition: ‘importer’ means any natural or legal person established within the Union who places a product from a third country on the Union market; statements:
‘supplier’ means a manufacturer established in the Union, an authorised representative of a manufacturer who is not established in the Union, or an importer, who places a product on the Union market;
output: True

User Input:
term: {term}
definition: {definition}
statements:
{sentences}
<<\SYS>>
[\INST]
"""

In [85]:
prompt = PromptTemplate(template=llama_template_term_comparison, input_variables=["term", "definition", "sentences"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [86]:
term = "C1 tyres"
definition = """
Tyres shall be classified as follows: (a)   class C1 tyres— tyres designed primarily for vehicles of categories M1, N1, O1 and O2; (b)   class C2 tyres— tyres designed primarily for vehicles of categories M2, M3, N, O3 and O4 with a load capacity index in single formation ≤ 121 and the speed category symbol ≥ ‘N’; (c)   class C3 tyres— tyres designed primarily for vehicles of categories M2, M3, N, O3 and O4 with one of the following load capacity indices:"""

sentences = """
In order to meet the challenge of reducing the CO2 emissions of road transport, it is appropriate that Member States, in cooperation with the Commission, provide for incentives to innovate with regard to fuel‐efficient and safe C1 tyres, C2 tyres and C3 tyres.
Currently, tyre labels are required for tyres for cars (C1 tyres) and vans (C2 tyres) but not for heavy‐duty vehicles (C3 tyres). C3 tyres consume more fuel and cover more kilometres per year than C1 tyres or C2 tyres, and therefore the potential to reduce fuel consumption and greenhouse gas emissions from heavy‐duty vehicles is significant. Therefore, C3 tyres should be included in the scope of this Regulation. Including C3 tyres fully in the scope of this Regulation is also in line with Regulation (EU) 2018/956 of the European Parliament and of the Council (8), which provides for the monitoring and reporting of CO2 emissions from and fuel consumption of new heavy‐duty vehicles, and with Regulation (EU) 2019/1242 of the European Parliament and of the Council (9), which sets CO2 emission performance standards for new heavy‐duty vehicles.
Suppliers shall ensure that C1 tyres, C2 tyres and C3 tyres that are placed on the market are accompanied free of charge: (a) for each individual tyre, by a tyre label, in the form of a sticker, that complies with the requirements set out in Annex II, indicating the information and class for each of the parameters set out in Annex I, and by a product information sheet; or (b) for each batch of one or more identical tyres, by a printed tyre label that complies with the requirements set out in Annex II, indicating the information and class for each of the parameters set out in Annex I, and by a product information sheet.
"""

In [87]:
llama_response = llm_chain.run({"term": term, "definition": definition, "sentences": sentences})
llama_response

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'Output: True'

In [88]:
# Negative Scenario
term_1 = "manufacturer"
definition_1 = """
‘manufacturer’ means any natural or legal person who manufactures a product or has a product designed or manufactured, and markets that product under its name or trademark;
"""

manufacturer_sentences = """
manufacturer destroyed tyres to optimise all parameters beyond the current standards
"""

In [89]:
llama_response_1 = llm_chain.run({"term": term_1, "definition": definition_1, "sentences": manufacturer_sentences})

In [90]:
llama_response_1

'output: False'

In [91]:
#Negative Scenario:
term_1 = "manufacturer"
definition_1 = """
‘manufacturer’ means any natural or legal person who manufactures a product or has a product designed or manufactured, and markets that product under its name or trademark;
"""

manufacturer_sentences = """
Traffic noise is a significant nuisance and has a harmful effect on health. Regulation (EC) No 661/2009 lays down minimum requirements for the external rolling noise of tyres. Technological developments make it possible to reduce external rolling noise significantly beyond those minimum requirements. To reduce traffic noise, it is therefore appropriate to update the provisions on the labelling of tyres to encourage manufacturer to purchase tyres with lower external rolling noise by providing them with harmonised information on the external rolling noise parameter.
"""

In [93]:
llama_response_2 = llm_chain.run({"term": term_1, "definition": definition_1, "sentences": manufacturer_sentences})
llama_response_2

'True'

In [203]:
llama_template_term_comparison_1 = """
<s>[INST]<<SYS>>
Act as a Lawyer drafting European Legislative documents to be published on Eur-Lex website.
Verify the term holds the definition in the context of the statements. Reply with only one word: True or False.

Examples:
term: importer
definition: ‘importer’ means any natural or legal person established within the Union who places a product from a third country on the Union market;
statements: ‘supplier’ means a manufacturer established in the Union, an authorised representative of a manufacturer who is not established in the Union, or an importer, who places a product on the Union market;
output: True

term: manufacturer
definition: ‘manufacturer’ means any natural or legal person who manufactures a product or has a product designed or manufactured, and markets that product under its name or trademark;
statements: Traffic noise is a significant nuisance and has a harmful effect on health. Regulation (EC) No 661/2009 lays down minimum requirements for the external rolling noise of tyres. Technological developments make it possible to reduce external rolling noise significantly beyond those minimum requirements. To reduce traffic noise, it is therefore appropriate to update the provisions on the labelling of tyres to encourage manufacturer to purchase tyres with lower external rolling noise by providing them with harmonised information on the external rolling noise parameter.
output: False

term: {term}
definition: {definition}
statements: {statements}
output:
<</SYS>>
[/INST]
"""

In [204]:
prompt = PromptTemplate(template=llama_template_term_comparison_1, input_variables=["term", "definition", "statements"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [208]:
llama_response_2 = llm_chain.run({"term": term, "definition": definition, "statements": statements})
llama_response_2

'True or False:\n\nterm: importer\ndefinition: ‘importer’ means any natural or legal person established within the Union who places a product from a third country on the Union market;\nstatements: ‘supplier’ means a manufacturer established in the Union, an authorised representative of a manufacturer who is not established in the Union, or an importer, who places a product on the Union market;\noutput: True\n\nterm: manufacturer\ndefinition: ‘manufacturer’ means any natural or legal person who manufactures a product or has a product designed or manufactured, and markets that product under its name or trademark;\nstatements: Traffic noise is a significant nuisance and has a harmful effect on health. Regulation (EC) No 661/2009 lays down minimum requirements for the external rolling noise of tyres. Technological developments make it possible to reduce external rolling noise significantly beyond those minimum requirements. To reduce traffic noise, it is therefore appropriate to update the

# New Prompts

In [252]:
llama_template_term_comparison = """
<s>[INST]<<SYS>>
Act as a Lawyer drafting European Legislative documents to be published on Eur-Lex website.
Verify the term holds the definition in the context of the statements. Reply with only one word: True or False.

Examples:
term: manufacturer
definition: ‘manufacturer’ means any natural or legal person who manufactures a product or has a product designed or manufactured, and markets that product under its name or trademark;
statements: ‘manufacturer’ means a consumer, fleet manager or road transport undertaking that buys or is expected to buy a tyre;
output: False

term: importer
definition: ‘importer’ means any natural or legal person established within the Union who places a product from a third country on the Union market;
statements: ‘supplier’ means a manufacturer established in the Union, an authorised representative of a manufacturer who is not established in the Union, or an importer, who places a product on the Union market;
output: True
<</SYS>>

User Input:
term: {term}
definition: {definition}
statements: {statements}
output:
[/INST]
"""

In [238]:
term = "custom authorities"
definition = """
"customs authorities" means the customs administrations of the Member States responsible for applying the customs legislation and any other authorities empowered under national law to apply certain customs legislation;"""
statements = """
Member States shall designate customs authorities, one or more market surveillance authorities or any other authority in their territory as the authorities in charge of the control on products entering the Union market.
customs authorities at the first point of entry have reason to believe that those products are not compliant with applicable Union law or present a risk, they shall transmit all relevant information to the competent customs office of destination.
"""

In [253]:
term = "custom authorities"
definition = """
"customs authorities" means the customs administrations of the Member States responsible for applying the customs legislation and any other authorities empowered under national law to apply certain customs legislation;"""
statements = """
Where the declaration or notification is lodged, the application is submitted, or information is provided, by a customs authorities of the person concerned, as referred to in Article 18, that customs authorities shall also be bound by the obligations set out in the first subparagraph of this paragraph.
"""

# "custom authorities" means any person appointed by another person to carry out the acts and formalities required under the customs legislation in his or her dealings

In [254]:
prompt = PromptTemplate(template=llama_template_term_comparison, input_variables=["term", "definition", "statements"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [257]:
llama_response = llm_chain.run({"term": term, "definition": definition, "statements": statements})
llama_response

'Great! I\'m ready to help you with that. Please provide the term you would like me to verify, and I will reply with "True" or "False" based on the definition provided.'

In [256]:
sections = llama_response.split("\noutput:")
sections[3].strip()

IndexError: ignored

In [177]:
term = "non-compliance"
definition = """
‘non-compliance’ means any failure to comply with any requirement under the Union harmonisation legislation or under this Regulation"""
statements = """
The costs referred to in paragraph 1 of this Article may include the costs of carrying out testing, the costs of taking measures in accordance with Article 28(1) and (2), the costs of storage and the costs of activities relating to products that are found to be non-compliant and are subject to corrective action prior to their release for free circulation or their non-compliance.
"""

In [198]:
llama_response_3 = llm_chain.run({"term": term, "definition": definition, "statements": statements})
llama_response_3

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'Sure, I\'d be happy to help! Please provide the term you would like me to verify, and I will respond with "True" or "False" based on the definition provided.'

In [190]:
# Split the text into sections based on the "output:" tag
sections = llama_response_3.split("\noutput:")

In [191]:
# Initialize a dictionary to store term-output pairs
term_output_dict = {}

# Iterate through the sections to extract term and output
for i in range(0, len(sections) - 1, 2):
    term_section = sections[i]
    output_section = sections[i + 1]

    # Extract the term
    term = term_section.split("term:")[1].strip()

    # Extract the output value (True or False)
    output_value = output_section.strip().lower() == "true"

    # Store the term-output pair in the dictionary
    term_output_dict[term] = output_value

# Print the dictionary of term-output pairs
print(term_output_dict)

{'manufacturer\ndefinition: Any natural or legal person who manufactures a product or has a product designed or manufactured, and markets that product under its name or trademark.\nstatements: A consumer, fleet manager, or road transport undertaking that buys or is expected to buy a tyre.': False, 'non-compliance\ndefinition: Any failure to comply with any requirement under the Union harmonisation legislation or under this Regulation.\nstatements: The costs referred to in paragraph 1 of this Article may include the costs of carrying out testing, the costs of taking measures in accordance with Article 28(1) and (2), the costs of storage, and the costs of activities relating to products that are found to be non-compliant and are subject to corrective action prior to their release for free circulation or their non-compliance.': True}


In [194]:
sections[3].strip()

'True'